**Table of contents**<a id='toc0_'></a>

- [Importing Libraries](#toc1_)
- [Configuring Visualization Parameters](#toc2_)
- [Configuring Other Notebook Parameters](#toc3_)
- [Pre-installing Custom Functions](#toc4_)
- [Practicing in Stages](#toc5_)
  - [Preprocessing](#toc5_1_)

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


# <a id='toc1_'></a>[Importing Libraries](#toc0_)


In [1]:
import pydicom

In [2]:
# import numpy as np
import pandas as pd

# import torch
# from torchvision.utils import make_grid

In [3]:
import matplotlib as mpl
# import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

In [4]:
# from datetime import datetime
# from functools import wraps
# import itertools
# import math
# import random
import reprlib
# from scipy import ndimage
import sys

# <a id='toc2_'></a>[Configuring Visualization Parameters](#toc0_)


In [5]:
%matplotlib inline

In [6]:
XINHUI = "#7a7374"
XUEBAI = "#fffef9"
YINBAI = "#f1f0ed"
YINHUI = "#918072"

figure_size = (16, 9)

In [7]:
custom_params = {
    "axes.axisbelow": True,
    "axes.edgecolor": YINBAI,
    "axes.facecolor": XUEBAI,
    "axes.grid": True,
    "axes.labelcolor": XINHUI,
    "axes.spines.right": False,
    "axes.spines.top": False,
    "axes.titlecolor": XINHUI,
    "figure.edgecolor": YINBAI,
    "figure.facecolor": XUEBAI,
    "grid.alpha": 0.8,
    "grid.color": YINBAI,
    "grid.linestyle": "--",
    "grid.linewidth": 1.2,
    "legend.edgecolor": YINHUI,
    "patch.edgecolor": XUEBAI,
    "patch.force_edgecolor": True,
    "text.color": XINHUI,
    "xtick.color": YINHUI,
    "ytick.color": YINHUI,
}

mpl.rcParams.update(custom_params)

# <a id='toc3_'></a>[Configuring Other Notebook Parameters](#toc0_)


In [8]:
reprlib_rules = reprlib.Repr()
reprlib_rules.maxother = 250

# <a id='toc4_'></a>[Pre-installing Custom Functions](#toc0_)


In [9]:
sys.path.append("../")

In [10]:
from Modules import *

# <a id='toc5_'></a>[Practicing in Stages](#toc0_)

## <a id='toc5_1_'></a>[Preprocessing](#toc0_)


In [11]:
labels = pd.read_csv(
    "../Datasets/Kaggle - RSNA Pneumonia Detection Challenge/stage_2_train_labels.csv"
)


#dataframe = DataFrame_Generator(item for item in labels.columns.values)
dataframe = DataFrame_Generator("epoch", "loss", "accuracy", "validation loss",
                                "validation accuracy")
for i in range(10):
    print(labels.iloc[i].values)
    #dataframe.updater(labels.iloc[i].values)

tabulation = Form_Generator()
tabulation.heading_printer("")

statements = [""]
tabulation.statement_generator(statements)

df_table = dataframe.tabulation()
tabulation.dataframe_generator(df_table)

['0004cfab-14fd-4e49-80ba-63a80b6bddd6' nan nan nan nan 0]
['00313ee0-9eaa-42f4-b0ab-c148ed3241cd' nan nan nan nan 0]
['00322d4d-1c29-4943-afc9-b6754be640eb' nan nan nan nan 0]
['003d8fa0-6bf1-40ed-b54c-ac657f8495c5' nan nan nan nan 0]
['00436515-870c-4b36-a041-de91049b9ab4' 264.0 152.0 213.0 379.0 1]
['00436515-870c-4b36-a041-de91049b9ab4' 562.0 152.0 256.0 453.0 1]
['00569f44-917d-4c86-a842-81832af98c30' nan nan nan nan 0]
['006cec2e-6ce2-4549-bffa-eadfcd1e9970' nan nan nan nan 0]
['00704310-78a8-4b38-8475-49f4573b2dbb' 323.0 577.0 160.0 104.0 1]
['00704310-78a8-4b38-8475-49f4573b2dbb' 695.0 575.0 162.0 137.0 1]


    +-----------+
    | Statement |
    +-----------+
    |           |
    +-----------+
    +---------+--------+------------+-------------------+
    | epoch   | loss   | accuracy   | validation loss   |
    |---------+--------+------------+-------------------+
    +---------+--------+------------+-------------------+
    … +-----------------------+
    … | validation acc

In [12]:
labels = labels.drop_duplicates("patientId")
labels.head(10)

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1
10,008c19e8-a820-403a-930a-bc74a4053664,NaN,NaN,NaN,NaN,0
11,009482dc-3db5-48d4-8580-5c89c4f01334,NaN,NaN,NaN,NaN,0
